In [1]:
from sqlalchemy import create_engine
import pandas as pd

#### Crear el "engine" pasando la dirección de la db

In [2]:
engine = create_engine('postgresql://celia@localhost:5432/mytestdb')
engine

Engine(postgresql://celia@localhost:5432/mytestdb)

#### Hacer la query especificando el "engine" que se desea usar

In [30]:
df_customer = pd.read_sql_query('select * from "Customer"',con=engine)

In [20]:
df_customer.to_json('/tmp/test.json')

In [22]:
json_df  = pd.read_json('/tmp/test.json')
json_df

,Birthday,CustomerCode,Gender
0,971136000000,1,F
1,1421020800000,2,M
2,1389312000000,3,F
3,1294617600000,4,F


In [37]:
json_df['Birthday'] = pd.to_datetime(json_df['Birthday'], unit='ns')
json_df

,Birthday,CustomerCode,Gender
0,1970-01-01 00:16:11.136000,1,F
1,1970-01-01 00:23:41.020800,2,M
2,1970-01-01 00:23:09.312000,3,F
3,1970-01-01 00:21:34.617600,4,F


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 3
Data columns (total 3 columns):
CustomerCode    4 non-null int64
Birthday        4 non-null object
Gender          4 non-null object
dtypes: int64(1), object(2)
memory usage: 128.0+ bytes


In [5]:
df.describe()

,CustomerCode
count,4.000000
mean,2.500000
std,1.290994
min,1.000000
25%,1.750000
50%,2.500000
75%,3.250000
max,4.000000


Link a [Pandas NB](Starting%20with%20pandas.ipynb) para ver join, merge, append, etc

#### Agregando un nuevo registro a nuestra tabla "Customer" con pandas

In [25]:
new_df = pd.DataFrame([[4, pd.datetime(2011, 1, 10), 'F']], columns=df.columns)
new_df

,CustomerCode,Birthday,Gender
0,4,2011-01-10,F


In [26]:
new_df.to_sql('Customer', engine, if_exists='append', index=None)

#### Agregando una nueva tabla a nuestra db desde pandas

In [29]:
new_table = pd.DataFrame([], columns=['WineCode', 'Type', 'Vintage'])
new_table

,WineCode,Type,Vintage


In [30]:
new_table.to_sql('Wine', engine, index=None)

#### Ahora hagamos lo mismo con sqlalchemy

In [40]:
from sqlalchemy import MetaData, types
from sqlalchemy import Table, Column

In [35]:
metadata = MetaData()

In [41]:
time = Table('Time', metadata,
    Column('TimeCode', types.Integer, primary_key=True),
    Column('Date', types.DateTime, nullable=False),
 )

In [42]:
metadata.create_all(engine)

### Agregando elementos a Wine con pandas

In [10]:
data = [[1, 'White', 2000],
        [2, 'red', 2015],
        [3, 'rose', 2014]]

In [11]:
new_df = pd.DataFrame(data, columns=df_wine.columns)

In [12]:
new_df.to_sql('Wine', engine, if_exists='append', index=None)

In [13]:
df_wine = pd.read_sql_query('select * from "Wine"',con=engine)
df_wine

,WineCode,Type,Vintage
0,1,White,2000
1,2,red,2015
2,3,rose,2014


In [32]:
data = [[1, 'White', pd.datetime(2000, 10, 10)],
        [2, 'red', pd.datetime(2010, 9, 9)],
        [3, 'rose', pd.datetime(2011, 9, 9)]]

In [36]:
new_df = pd.DataFrame(data, columns=df_wine.columns)
new_df['Vintage']

0   2000-10-10
1   2010-09-09
2   2011-09-09
Name: Vintage, dtype: datetime64[ns]

In [46]:
new_df.to_json('/tmp/lero.json', date_unit='ns')

In [47]:
json_demo = pd.read_json('/tmp/lero.json')
jso

,Type,Vintage,WineCode
0,White,971136000000000000,1
1,red,1283990400000000000,2
2,rose,1315526400000000000,3


In [52]:
new_df

,WineCode,Type,Vintage
0,1,White,2000-10-10
1,2,red,2010-09-09
2,3,rose,2011-09-09


In [48]:
json_demo['Vintage'] = pd.to_datetime(json_demo['Vintage'], unit='ns')

In [49]:
json_demo

,Type,Vintage,WineCode
0,White,2000-10-10,1
1,red,2010-09-09,2
2,rose,2011-09-09,3


In [55]:
new_df

,WineCode,Type,Vintage
0,1,White,2000-10-10
1,2,red,2010-09-09
2,3,rose,2011-09-09


In [65]:
json_demo.columns.values

array([u'Type', u'Vintage', u'WineCode'], dtype=object)

In [66]:
json_demo.values

array([[u'White', Timestamp('2000-10-10 00:00:00'), 1],
       [u'red', Timestamp('2010-09-09 00:00:00'), 2],
       [u'rose', Timestamp('2011-09-09 00:00:00'), 3]], dtype=object)

In [67]:
pd.merge(json_demo, new_df, on=list(json_demo.columns.values), how='outer')

,Type,Vintage,WineCode
0,White,2000-10-10,1
1,red,2010-09-09,2
2,rose,2011-09-09,3


In [82]:
pd.Series?

In [83]:
score = pd.Series([10, 9, 8], name='score')
score

0    10
1     9
2     8
Name: score, dtype: int64

In [96]:
out = pd.concat([json_demo, score], axis=1)
new_row = pd.DataFrame([[4, 'espumeante',pd.datetime(2000,2,2)]], columns=new_df.columns)

In [101]:
append_df = new_df.append(new_row)

In [105]:
append_df.to_sql('Wine', engine, if_exists='append', index=None)